# 4장 Lists

## 4.1 List notation

List 자료구조는 functional programming에서 가장 중요한 자료구조이다. List는 매우 많은 유용한 연산을 가지고 있다.

a 자료형으로 이루어진 list는 [a]로 표기한다.

    [undefined,undefined] :: [a]
    [sin,cos,tan]         :: Floating a => [a -> a]
    [[1,2,3],[4,5]]       :: Num a => [[a]]
    ["tea","for",2]          not valid

사실 [a]의 list 표기법은 1:2:3:[]의 축약이다. (:) :: a -> [a] -> [a] 는 list를 만드는 연산자이며 'cons'라 부른다. (:)는 두 인자 모두에 대해서 non-strict로 동작한다. 따라서 undefined : undefined도 가능하다.

empty list []도 list constructor이다. List는 Haskell data ypte에서 아래와 같이 선언할 수 있다.

    data List a = Nil | Cons a (List a)

위 정의에 따르면 모든 list는 다음의 3가지 형태 중에 하나임.

* undefined :: [a]
* [] :: [a]
* x:xs where x :: a and xs :: [a]

그 결과 3가지 list가 존재한다.

* finite list: (:)과 []로 만들어짐. 1:2:3:[]
* partial list: (:)가 undefined로 만들어짐. 1:2:3:undefined
* infinite list: (:)로만 만들어짐. [1..]

## 4.2 Enumerations

    [m..n]
    [m..]
    [m,n..p]
    [m,n..]

In [1]:
[0..10]
[0,5..27]
['a'..'z']

[0,1,2,3,4,5,6,7,8,9,10]

[0,5,10,15,20,25]

"abcdefghijklmnopqrstuvwxyz"

## 4.3 List comprehensions

In [2]:
[x*x | x <- [1..5]]
[x*x | x <- [1..5], odd x]
[(i,j) | i <- [1..5], even i, j <- [i..5]]
[x | xs <- [[(3,4)],[(5,4),(3,2)]], (3,x) <- xs]

triads :: Int -> [(Int,Int,Int)]
triads n = [(x,y,z) | x <- [1..n], y <- [1..n],
                      z <- [1..n], x*x+y*y==z*z]

triads 15

[1,4,9,16,25]

[1,9,25]

[(2,2),(2,3),(2,4),(2,5),(4,4),(4,5)]

[4,2]

[(3,4,5),(4,3,5),(5,12,13),(6,8,10),(8,6,10),(9,12,15),(12,5,13),(12,9,15)]

In [3]:
map' f xs = [f x | x <- xs]
filter' p xs = [x | x <- xs, p x]
concat' xss = [x | xs <- xss, x <- xs]

map' (+1) [2,3,4] ==[(+1) x | x <- [2,3,4]]

True

The translation rules are:

    [e | True]        = [e]
    [e | q]          = [e | q, True]
    [e | b, Q]       = if b then [e | Q] else []
    [e | p <- xs, Q] = let ok p = [e | Q]
                           ok _ = []
                       in concat (map ok xs)

'_' is a 'don't care' pattern (wild card)

## 4.4 Some basic operations

In [4]:
null' :: [a] -> Bool
null' []     = True
null' (x:xs) = False

head' :: [a] -> a
head' (x:xs) = x

tail' :: [a] -> [a]
tail' (x:xs) = xs


last' [x]    = x
last' (_:xs) = last xs

## 4.5 Concatenation

    (++) :: [a] -> [a] -> [a]
    [] ++ ys     = ys
    (x:xs) ++ ys = x:(xs ++ ys)

## 4.6 concat, map and filter

    concat :: [[a]] -> [a]
    concat [] =[]
    concat (xs:xss) = xs ++ concat xss
    
    map :: (a -> b) -> [a] -> [b]
    map f []     = []
    map f (x:xs) = f x:map f xs
    
    filter :: (a -> Bool) -> [a] -> [a]
    filter p []     = []
    filter p (x:xs) = if p x then x:filter p xs
                      else filter p xs

map은 다음과 같은 특성을 가지며 이 특성은 map의 functor laws라 불린다.

    map id      = id
    map (f . g) = map f . map g

Haskell은 type class Functor를 가지고 있다.

    class Functor f where
      fmap :: (a -> b) -> f a -> f b

Functor는 List 외의 다른 자료구조에도 일반화할 수 있다. 아래는 Tree에 대한 Functor instance 예임.

    data Tree a = Tip a | Fork (Tree a) (Tree a)
    
    instance Functor Tree where
      fmap f (Tip x) = Tip (f x)
      fmap f (Fork u v) = Fork (fmap f u) (fmap f v)

map은 list의 fmap과 동일하다.

In [5]:
fmap (+1) [2,3,4]
map (+1) [2,3,4]

[3,4,5]

[3,4,5]

Functor에서 보았듯이 특정 자료구조에 의존하지 않는 일반화할 수 있는 것들이 나오면 이것은 적합한 type class로 만들어질 수 있다.

map에 대한 다른 규칙들을 보면

    f . head       = head . map f
    map f . tail   = tail . map f
    map f . concat = concat . map (map f)
    
이 중 첫번째는 strict function에 대해서만 만족한다. 만약 []에 적용을 해본 다면

    f (head []) = head (map f []) = head []
    
head []는 undefined 이므로, 이 등식을 참으로 하기 위해서는 f가 strict이어야 한다.

이 밖에도 몇 가지 법칙들을 더 보면

    map f . reverse = reverse . map f
    
    concat . map concat = concat . concat
    
    filter p . map f = map f . filter (p . f)

## zip and zipWith

    zip :: [a] -> [b] -> [(a,b)]
    zip (x:xs) (y:ys) = (x,y): zip xs ys
    zip _ _ = []
    
    zip = zipWith (,)
    
haskell의 패턴매칭은 위에서 아래, 좌에서 우의 순서로 이루어진다. 따라서 아래와 같은 결과가 나온다.

    zip [] undefined = []
    zip undefined [] = undefined

list가 모든 원소에 대해서 증가하는 패턴인지 확인하는 nondec 함수

    nondec :: (Ord a) => [a] -> Bool
    nondec []    = True
    nondec [x]   = True
    nondec (x:y:xs) = (x <= y) && nondec (y:xs)
    
아래와 같이도 정의 가능
   
    nondec xs = and (zipWith (<=) xs (tail xs))

position은 xs에서 처음 나오는 x의 위치를 찾는다. x가 없는 경우 -1 반환

In [6]:
position :: (Eq a) => a -> [a] -> Int
position x xs = head ([j | (j,y) <- zip [0..] xs, y==x] ++ [-1])

position 3 [1..5]

2

## 4.8 Common words, completed

1.3의 commonWords 프로그램을 완성해보자. (직접 구현했던 것과 좀 다름)
span을 사용해서 카운팅된 단어는 제거하고 나머지에 대해서만 재귀수행하여 성능이 더 좋음.

    span :: (a -> Bool) -> [a] -> ([a],[a])
    span p []     = ([],[])
    span p (x:xs) = if p x then (x:ys,zs)
                    else ([],x:xs)
                    where (ys,zs) = span p xs

sort, merge도 직접 구현했는데, merge에 사용된 @은 as의 의미로 쓰임.

    -- (x:xs)를 xs'로 바인딩
    xs'@(x:xs)

sort [x] 패턴이 없는 경우 무한루프에 빠짐

    sort [x] = merge (sort []) (sort [x])

In [7]:
-- imports
import qualified Data.Char as Char
import qualified Data.List as List

type Text = [Char]
type Word = [Char]

-- map :: (a->b) -> [a] -> [b] from Prelude
-- toLower :: Char -> Char from Data.Char
-- map toLower :: Text -> Text

sortWords :: [Word] -> [Word]
sortWords = sort'

countRuns :: [Word] -> [(Int,Word)]
countRuns []     = []
countRuns (w:ws) = (1+length us,w):countRuns vs
                   where (us,vs) = span (==w) ws
                         
sortRuns :: [(Int,Word)] -> [(Int,Word)]
sortRuns = List.reverse . sort'

-- take :: Int -> [a] -> [a]

showRun :: (Int,Word) -> String
showRun (n,w) = w ++ ": " ++ show n ++ "\n"

sort' :: (Ord a) => [a] -> [a]
sort' [] =[]
sort' [x] = [x]
sort' xs = merge' (sort' ys) (sort' zs)
           where (ys,zs) = halve xs
halve xs = (take n xs, drop n xs)
            where n = length xs `div` 2

merge' :: (Ord a) => [a] -> [a] -> [a]
merge' [] ys = ys
merge' xs [] = xs
merge' xs'@(x:xs) ys'@(y:ys)
  | x <= y    = x:merge' xs ys'
  | otherwise = y:merge' xs' ys

-- map showRun :: [(Int,Word)] -> [String]

-- concat :: [[a]] -> [a] from Prelude
commonWords :: Int -> Text -> String
commonWords n = concat . map showRun . take n . 
                sortRuns . countRuns . sortWords . 
                words . map Char.toLower

sample = "cc ba aa ba cc cc"

putStrLn $ commonWords 2 sample

cc: 3
ba: 2